<a href="https://colab.research.google.com/github/barauna-lo/Colored_Noise/blob/main/2DPSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Código para Cálculo do Beta para um ruido 1/f 2D (imagem), R.R.Rosa-INPE-06/4/2022

In [ ]:
from sys import setdlopenflags
import matplotlib.image as mpimg
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as pl
 

image = mpimg.imread("clouds.png")
pl.imshow(mpimg.imread('clouds.png'),cmap='gray')

In [ ]:
#2DFFT
npix = image.shape[0]

fourier_image = np.fft.fftn(image)
fourier_amplitudes = np.abs(fourier_image)**2

kfreq = np.fft.fftfreq(npix) * npix
kfreq2D = np.meshgrid(kfreq, kfreq)
knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

knrm = knrm.flatten()
fourier_amplitudes = fourier_amplitudes.flatten()

kbins = np.arange(0.5, npix//2+1, 1.)
kvals = 0.5 * (kbins[1:] + kbins[:-1])
Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                     statistic = "mean",
                                     bins = kbins)
Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

pl.loglog(kvals, Abins)
pl.xlabel("$k$")
pl.ylabel("$P(k)$")

x=kvals
y=Abins

logx = np.log(x)
logy = np.log(y)
coeffs = np.polyfit(logx,logy,deg=3)
poly = np.poly1d(coeffs)

yfit = lambda x: np.exp(poly(np.log(x)))
pl.loglog(x,yfit(x))

#calculo do slope (beta do PSD)
slope, intercept = np.polyfit(np.log(x),np.log(yfit(x)),1)
pl.title('PSD from 2DFFT with beta= {}'.format(slope))
pl.grid(True)

pl.show()
pl.tight_layout()
pl.savefig("cloud_power_spectrum.png", dpi = 300, bbox_inches = "tight")

In [ ]:
from PIL import Image 
fname = r'clouds.png'
image = Image.open(fname).convert("L") 
pl.imshow(image, cmap='gray') 
pl.show()
print(slope)

In [ ]:
def getHistogramSlices(mat):
    freq = np.fft.fftfreq(mat.shape[0])
    p = (freq > 1e-12)
    betas= []
    for i in range(mat.shape[0]):
        psd = np.fft.fft(mat[i])
        psd = (psd*np.conj(psd)).real
        b,_ = np.polyfit(np.log(freq[p]),np.log(psd[p]),deg=1)
        betas.append(-b)

    freq = np.fft.fftfreq(mat.shape[1])
    for i in range(mat.shape[1]):
        psd = np.fft.fft(mat[:,i])
        psd = (psd*np.conj(psd)).real
        b,_ = np.polyfit(np.log(freq[p]),np.log(psd[p]),deg=1)
        betas.append(-b)
    return betas

In [ ]:
cloudImg = np.array(image.getdata()).reshape(image.size[0], image.size[1])
betas = getHistogramSlices(cloudImg) 
pl.figure()
pl.hist(betas,label='avg: '+str(np.round(np.average(betas),3)))
pl.xlabel(r"$\beta$",fontsize=14)
pl.ylabel('Quantity',fontsize=14)
pl.legend()
pl.show()

# Gerando ruído

In [ ]:
from numpy.random import normal
import numpy as np

def cNoise(beta,shape=(1024,),std=0.001, maxCorrections=10,maxAvgError=0.01, eta=0.6):
    '''
       Wrote by: Rubens Andreas Sautter (2021)
       
       An parameter of correction has been used (s):
       	FFT(f(w)) = gauss(0,std) * (1/w^beta)^(beta*s/2) 
       
        Frequencies are measured in multidimensional space by the frequency euclidian distance.
        
       =====================================================================================
       beta (float) - the colored noise decay (0-white noise, 1-pink noise, 2- red noise)
       shape (tuple) - the output shape
       std  (float) standard deviation of the spectrum gaussian function (see reference)
       maxCorrections (int) - maximum number of iteractions of the process of decay correction 
       maxAvgError (float) - target error of the spectrum decay
       eta (float) - optimization parameter, large eta can sometimes not reach the minimum
       		small eta is slower (like the gradient descent eta)
       		
       		* For beta = [0,2], eta>=0.6 seems to converge
       		      beta - 3, eta <= 0.6 seems to converge
       
       =====================================================================================
       Inspired by:
      http://articles.adsabs.harvard.edu//full/1995A%26A...300..707T/0000707.000.html
    '''
    dimension = []
    for index,dsize in enumerate(shape):
        dimension.append(np.fft.fftfreq(dsize).tolist())
    dimension = tuple(dimension)
    d = float(len(dimension))
    
    freqs = np.power(np.sum(np.array(np.meshgrid(*dimension,indexing='ij'))**2,axis=0),1/2)*np.sqrt(2)/4
    
    #Sampling gaussian with sandard deviation varying according to frequency
    ftSample = normal(loc=0,scale=std,size=shape) + 1j*normal(loc=0,scale=std,size=shape)
    
    # Setting the scale [0,2pi]
    freqs = np.pi*freqs
    not0Freq = (np.abs(freqs)>1e-15)
    positiveFreq = (freqs>1e-15)
    
    decayCorrectionL = []
    errorL = []
    
    # Building the first spectrum trial
    decayCorrection = np.sqrt(2)**(d-1)
    scaling = (freqs[not0Freq]+0j)**(-(beta*decayCorrection)/2)
    generatedSpectrum = ftSample.copy()
    generatedSpectrum[not0Freq] = (ftSample[not0Freq]*scaling)
    spsd = np.sum(np.abs(generatedSpectrum))
    out = np.fft.ifftn(generatedSpectrum).real
    # zero avg
    ftSample[0] = 0.0
    
    # one dimensional noise does not require corrections
    if len(dimension)==1:
        return out
    
    #measuring the average beta
    betas = []
    for i in range(len(out)):
        series = out[i,...]	
        # multidimensional slice
        if(len(dimension)>2):
            for j in range(len(dimension)-2):
                series = series[0]
        psd = np.fft.fft(series)
        psd = np.real(psd*np.conj(psd))
        lfreqs = np.fft.fftfreq(len(series))
        fPSD = psd[lfreqs>0.0]
        fFreqs = lfreqs[lfreqs>0.0]
        fit = np.polyfit(np.log(fFreqs),np.log(fPSD),deg=1)
        betas.append(-fit[0])
    

    	
    # measuring the error 
    smallCorrection = beta-np.average(betas)
    
    #including in the list
    decayCorrectionL.append(decayCorrection)
    errorL.append(smallCorrection)
    
    countCycles = 0
    # rebuilding the spectrum
    while np.abs(smallCorrection)>maxAvgError:
        decayCorrection += smallCorrection*eta
        scaling = (freqs[not0Freq]+0j)**(-(beta*decayCorrection)/2)
        generatedSpectrum = ftSample.copy()
        generatedSpectrum[not0Freq] = (ftSample[not0Freq]*scaling)
        spsd = np.sum(np.abs(generatedSpectrum))
        out = np.fft.ifftn(generatedSpectrum).real

        #measuring the average beta
        betas = []
        for i in range(len(out)):
            series = out[i,...]
            # multidimensional slice
            if(len(dimension)>2):
                for j in range(len(dimension)-2):
                    series = series[0]
            psd = np.fft.fft(series)
            psd = np.real(psd*np.conj(psd))
            lfreqs = np.fft.fftfreq(len(series))
            fPSD = psd[lfreqs>0.0]
            fFreqs = lfreqs[lfreqs>0.0]
            fit = np.polyfit(np.log(fFreqs),np.log(fPSD),deg=1)
            betas.append(-fit[0])
        	
        # measuring the error
        smallCorrection = beta-np.average(betas)
        
        print("Noise error - ", smallCorrection)
        decayCorrectionL.append(decayCorrection)
        errorL.append(smallCorrection)
    	
        countCycles = countCycles+1
        if countCycles>maxCorrections:
            break
            
    
    # resampling with the best decay
    errorL = np.abs(errorL)
    print("Best decay constant:", decayCorrectionL[np.argmin(errorL)]," Error: ",errorL[np.argmin(errorL)])
    decayCorrection = decayCorrectionL[np.argmin(errorL)]
    scaling = (freqs[not0Freq]+0j)**(-(beta*decayCorrection)/2)
    generatedSpectrum = ftSample.copy()
    generatedSpectrum[not0Freq] = (ftSample[not0Freq]*scaling)
    spsd = np.sum(np.abs(generatedSpectrum))
    out = np.fft.ifftn(generatedSpectrum).real

    # normalizing
    out = out / np.max(np.abs(out))
    return out

In [ ]:
size = 1000
wn = cNoise(0,(size,size),maxCorrections=100,maxAvgError=0.001, eta=0.05)
pn = cNoise(1,(size,size),maxCorrections=100,maxAvgError=0.001, eta=0.05)
rn = cNoise(2,(size,size),maxCorrections=100,maxAvgError=0.001, eta=0.05)
pl.figure()
pl.imshow(wn,cmap='gray')
pl.show()

In [ ]:
#2DFFT

file_cloud = 'cloud_power_spectrum.png'

def fft2DPSD(image,file_name):
  npix = image.shape[0]
  fourier_image = np.fft.fftn(image)
  fourier_amplitudes = np.abs(fourier_image)**2

  kfreq = np.fft.fftfreq(npix) * npix
  kfreq2D = np.meshgrid(kfreq, kfreq)
  knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

  knrm = knrm.flatten()
  fourier_amplitudes = fourier_amplitudes.flatten()

  kbins = np.arange(0.5, npix//2+1, 1.)
  kvals = 0.5 * (kbins[1:] + kbins[:-1])
  Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                      statistic = "mean",
                                      bins = kbins)
  Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

  #FIGURE

  figure(figsize=(3, 3), dpi=100)
  pl.loglog(kvals, Abins)
  pl.xlabel("$k$")
  pl.ylabel("$P(k)$")

  x=kvals
  y=Abins

  logx = np.log(x)
  logy = np.log(y)
  coeffs = np.polyfit(logx,logy,deg=3)
  poly = np.poly1d(coeffs)

  yfit = lambda x: np.exp(poly(np.log(x)))
  pl.loglog(x,yfit(x))

  #calculo do slope (beta do PSD)
  slope, intercept = np.polyfit(np.log(x),np.log(yfit(x)),1)
  pl.title(f'PSD from 2DFFT with β= {round(slope,3)}',fontsize=10)
  pl.grid(True)
  pl.ylim(1e5,1e11)
  pl.savefig(file_name, dpi = 300, bbox_inches = "tight")
  pl.show()
  pl.tight_layout()
  

In [ ]:

imagens = [image,wn,pn,rn]
save_has = ['PSD_cloud.png',
            'PSD_wn.png',
            'PSD_pn.png',
            'PSD_rn.png',]
for i in range(len(imagens)):
  fft2DPSD(imagens[i],save_has[i])

In [ ]:
color = ['gray','black','magenta','red']
his_title = ['Histogram for Cloud','Seeded β = 0','Seed β = 1','Seeded β = 2',]
his_save = ['HIST_cl.png','HIST_wn.png','HIST_pn.png','HIST_rn.png',]
def hist_plot(image):
  cloudImg = image
  figure(figsize=(3, 3), dpi=100)
  betas = getHistogramSlices(cloudImg) 
  #pl.figure()
  pl.hist(betas,label='avg: '+str(np.round(np.average(betas),3)),color=color[i],bins=30)
  pl.title(his_title[i],fontsize=10)
  pl.xlabel(r"$\beta$", fontsize=10)
  pl.ylabel('Quantity', fontsize=10)
  pl.legend()
  pl.savefig(his_save[i], dpi = 300, bbox_inches = "tight")  
  pl.show()

In [ ]:
for i in range(0,4):
  hist_plot(imagens[i])

# Extensão para os 4 casos

In [ ]:
titulos = ['Cloud','White Noise - β=0','Pink Noise - β=1','Red Noise - β=2']
save_fi = ['cloud_noise.png','white_noise.png','pink_noise.png','red_noise.png']


for i in range(len(imagens)):
  figure(figsize=(3, 3), dpi=100)
  #CLOUD
  pl.imshow(imagens[i],cmap='gray')
  pl.title(titulos[i],fontsize=10)  
  pl.axis('off')
  pl.savefig(save_fi[i])


In [ ]:
#Ziping Data for download📦
!zip -r /content/All_Files.zip /content

In [ ]:
  #2DFFT

image1 = mpimg.imread("white.png")
image2 = mpimg.imread("pink.png")
image3 = mpimg.imread("red.png")
image = mpimg.imread("clouds.png")




for i in range(len(imagens)):
  npix = imagens[i].shape[0]
  fourier_image = np.fft.fftn(imagens[i])
  fourier_amplitudes = np.abs(fourier_image)**2

  kfreq = np.fft.fftfreq(npix) * npix
  kfreq2D = np.meshgrid(kfreq, kfreq)
  knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

  knrm = knrm.flatten()
  fourier_amplitudes = fourier_amplitudes.flatten()

  kbins = np.arange(0.5, npix//2+1, 1.)
  kvals = 0.5 * (kbins[1:] + kbins[:-1])
  Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                      statistic = "mean",
                                      bins = kbins)
  Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

  pl.loglog(kvals, Abins)
  pl.xlabel("$k$")
  pl.ylabel("$P(k)$")

  x=kvals
  y=Abins

  logx = np.log(x)
  logy = np.log(y)
  coeffs = np.polyfit(logx,logy,deg=3)
  poly = np.poly1d(coeffs)

  yfit = lambda x: np.exp(poly(np.log(x)))
  pl.loglog(x,yfit(x))

  #calculo do slope (beta do PSD)
  slope, intercept = np.polyfit(np.log(x),np.log(yfit(x)),1)
  pl.title('PSD from 2DFFT with beta= {}'.format(slope))
  pl.grid(True)

  pl.show()
  pl.tight_layout()
  pl.savefig("cloud_power_spectrum.png", dpi = 300, bbox_inches = "tight")

In [ ]:
#2DFFT

npix = image1.shape[0]

fourier_image = np.fft.fftn(image1)
fourier_amplitudes = np.abs(fourier_image)**2

kfreq = np.fft.fftfreq(npix) * npix
kfreq2D = np.meshgrid(kfreq, kfreq)
knrm = np.sqrt(kfreq2D[0]**2 + kfreq2D[1]**2)

knrm = knrm.flatten()
fourier_amplitudes = fourier_amplitudes.flatten()

kbins = np.arange(0.5, npix//2+1, 1.)
kvals = 0.5 * (kbins[1:] + kbins[:-1])
Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                     statistic = "mean",
                                     bins = kbins)
Abins *= np.pi * (kbins[1:]**2 - kbins[:-1]**2)

pl.loglog(kvals, Abins)
pl.xlabel("$k$")
pl.ylabel("$P(k)$")

x=kvals
y=Abins

logx = np.log(x)
logy = np.log(y)
coeffs = np.polyfit(logx,logy,deg=3)
poly = np.poly1d(coeffs)

yfit = lambda x: np.exp(poly(np.log(x)))
pl.loglog(x,yfit(x))

#calculo do slope (beta do PSD)
slope, intercept = np.polyfit(np.log(x),np.log(yfit(x)),1)
pl.title('PSD from 2DFFT with beta= {}'.format(slope))
pl.grid(True)

pl.show()
pl.tight_layout()
pl.savefig("cloud_power_spectrum.png", dpi = 300, bbox_inches = "tight")

# 1D Graphs Generate

In [ ]:
# !pip install jupyternotify # Install de Jupyter Notify 
# %load_ext jupyternotify    # Loadgin Jupyter Notify
!pip install colorednoise
import colorednoise as cn
from matplotlib import mlab
from matplotlib import pylab as plt
import numpy as np
import pandas as pd
import tensorflow

In [ ]:
import colorednoise as cn
#from matplotlib import mlab
from matplotlib import pylab as plt
#import numpy as np

#input values
beta = 1         # the exponent: 0=white noite; 1=pink noise;  2=red noise (also "brownian noise")
samples = 2**16  # number of samples to generate (time series extension)

#Deffing some colores
beta = [0,1,2]
colors = ['black','magenta','red']

for i in range(len(beta)):
  A = cn.powerlaw_psd_gaussian(beta[i], samples)

  #Deffing the great figure size
  plt.figure(figsize=(6,3),dpi=800)

  #Ploting first subfiure
  #plt.subplot(1,2,1)
  plt.plot(A, color=colors[beta[i]], linewidth=1)
  plt.title('Colored Noise for β='+str(beta[i]))
  plt.xlabel('Samples (time-steps)')
  plt.ylabel('Amplitude(t)', fontsize='large')
  plt.xlim(1,32000)

  #Ploting second subfigure
  # plt.subplot(122)
  # spectrum, frequency = mlab.psd(A, NFFT=2**13)
  # plt.loglog(frequency,spectrum, color=colors[beta], linewidth=0.8)
  # plt.title('Power Spectral Density of A(t) with β='+str(beta))
  # plt.xlabel('Frequency')
  # plt.ylabel('Power Spectrum Density', fontsize='large')
  # plt.grid(True)
  plt.savefig("color_noide_beta="+str(beta[i])+".png")

#ploting the intire figure
plt.show()